In [6]:
import pandas as pd

In [7]:
qvals = pd.read_csv("/mnt/lab_data2/anusri/variant-scorer/src/output/caqtls_lcl_latest/lead_caQTL_variants.tsv.gz", sep="\t", header=0)
qvals["chr"] = "chr"+qvals["Chr"].astype(str) 
contain_values = qvals[~qvals['Alt'].str.contains('<CN')]
contain_values = contain_values[~contain_values['Alt'].str.contains(',')]
contain_values = contain_values[contain_values['Alt'].str.len()==1]
contain_values = contain_values[contain_values['Ref'].str.len()==1]
contain_values = contain_values.sort_values('Log10_BF').drop_duplicates(["chr", "Ref", "Alt", "Pos"], keep='last')

In [8]:
qvals.head()

,Peak,Chr,Pos,RsID,Ref,Alt,AF,Inside_Peak,P_Lead,Beta,SE,Log10_BF,chr
0,1,1,10352,rs555500075,T,TA,0.38240,1,0.565592,0.091926,0.055039,-1.165323,chr1
1,2,1,10177,rs367896724,A,AC,0.32395,2,0.021318,-0.141834,0.062422,-0.618151,chr1
2,3,1,875770,rs4970379,A,G,0.62500,2,0.044994,0.155446,0.057836,-0.232537,chr1
3,4,1,1142714,esv3584990,T,<CN0>,0.13485,0,0.202811,-0.089320,0.072935,-1.315789,chr1
4,5,1,930533,rs3128110,C,G,0.35435,0,0.099392,0.174020,0.040562,1.751829,chr1


In [13]:
sum(qvals["Log10_BF"]>14)

2862

In [3]:
qvals.head()

,Peak,Chr,Pos,RsID,Ref,Alt,AF,Inside_Peak,P_Lead,Beta,SE,Log10_BF,chr
0,1,1,10352,rs555500075,T,TA,0.38240,1,0.565592,0.091926,0.055039,-1.165323,chr1
1,2,1,10177,rs367896724,A,AC,0.32395,2,0.021318,-0.141834,0.062422,-0.618151,chr1
2,3,1,875770,rs4970379,A,G,0.62500,2,0.044994,0.155446,0.057836,-0.232537,chr1
3,4,1,1142714,esv3584990,T,<CN0>,0.13485,0,0.202811,-0.089320,0.072935,-1.315789,chr1
4,5,1,930533,rs3128110,C,G,0.35435,0,0.099392,0.174020,0.040562,1.751829,chr1


In [4]:
def get_data_for_ap(data_df, sig_thresh, nonsig_thresh, namef):
    best_sig = data_df[data_df["Log10_BF"]>sig_thresh]
    best_sig["label"] = 1
    
    # non-significant set inside a peak
    nonsig_df = data_df[data_df["Log10_BF"]<nonsig_thresh].reset_index()
    
    other_nonsigs = nonsig_df
    #other_nonsigs_inp = other_nonsigs[other_nonsigs["Inside_Peak"]>0]
    other_nonsigs_inp = other_nonsigs
    final_set_nonsigs = other_nonsigs_inp.sample(best_sig.shape[0]*50, random_state=1).reset_index(drop=True)
    final_set_nonsigs["label"] = 0
     
    alldf = pd.concat([final_set_nonsigs,best_sig])
    
    alldf = alldf[["chr", "Pos", "Ref", "Alt", "RsID", "label", "Beta"]]
    
    alldf.to_csv(namef, sep="\t", header=True, index=False)
    

In [5]:
get_data_for_ap(contain_values, 20, -1, "/mnt/lab_data2/anusri/enformer/caqtls/set_sig20/source1.tsv")

/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
